In [ ]:
from sklearn.cross_decomposition import CCA
import pandas as pd
import joblib
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from nilearn.signal import clean

In [ ]:
pop_cca = joblib.load('org_CCA_k25_lonely_full')

In [ ]:
pop_cca

# PREVENT-AD data 

In [ ]:
#HC data 
BL00_HC = pd.read_csv('HC_segmentation/csv/BL00_HC_left_right.csv', index_col=0)
FU12_HC = pd.read_csv('HC_segmentation/csv/FU12_HC_left_right.csv', index_col=0)
FU24_HC = pd.read_csv('HC_segmentation/csv/FU24_HC_left_right.csv', index_col=0)
FU36_HC = pd.read_csv('HC_segmentation/csv/FU36_HC_left_right.csv', index_col=0)
FU48_HC = pd.read_csv('HC_segmentation/csv/FU48_HC_left_right.csv', index_col=0)

In [ ]:
#DN data
BL00_DN = pd.read_csv('BL00_DN.csv', index_col=0)
FU12_DN = pd.read_csv('FU12_DN.csv', index_col=0)
FU24_DN = pd.read_csv('FU24_DN.csv', index_col=0)
FU36_DN = pd.read_csv('FU36_DN.csv', index_col=0)
FU48_DN = pd.read_csv('FU48_DN.csv', index_col=0)

In [ ]:
col_HC = list(BL00_HC.columns)[0:-4]

In [ ]:
col_HC.remove('Whole_hippocampal_body_left')
col_HC.remove('Whole_hippocampal_head_left')
col_HC.remove('Whole_hippocampus_left')
col_HC.remove('PSCID_left')

In [ ]:
len(col_HC)

In [ ]:
BL00_DN.shape

In [ ]:
col_DN = BL00_DN.columns

In [ ]:
BL00_HC[col_HC].shape

In [ ]:
#removing two participants without HC volumes based on index
BL00_DN = BL00_DN.iloc[np.where(BL00_DN.index.isin(BL00_HC[col_HC].index))]

# Deconfounding

In [ ]:
#retreiving fixed data (demographics, genetics, etc..) recorder at initial visit
EN00 = pd.read_csv(os.path.abspath('/Users/chloesavignac/_bzdok_lab_notebooks/Prevent-AD/internal/non-imaging-data/EL00.csv'), index_col=0)

In [ ]:
#setting indixes to participant ids
EN00 = EN00.set_index('PSCID_x',drop=False).rename(columns={'PSCID_x':'PSCID'})

In [ ]:
#verifying that each age entry is the same 
EN00[['Candidate_Age_x','Candidate_Age_y','Candidate_Age_x.1','Candidate_Age_y.1','Candidate_Age_x.2','Candidate_Age_y.2']]

In [ ]:
#keeping only one column for age
EN00['Age_months'] = EN00.Candidate_Age_x

In [ ]:
EN00['Age_years'] = EN00['Age_months']/12

In [ ]:
#we want the sex of each participant based on PSCID
infos = EN00[['CandID','PSCID','Sex', 'Age_years','father_dx_ad_dementia','mother_dx_ad_dementia','sibling_dx_ad_dementia','APOE']]
infos

In [ ]:
def deconf(beh):

    age = StandardScaler().fit_transform(beh['Age_years'].values[:, np.newaxis])  # Age at recruitment
    age2 = age ** 2
    sex = np.array(pd.get_dummies(beh['Sex']).values, dtype=np.int)  # Sex
    sex_x_age = sex * age
    sex_x_age2 = sex * age2
    
    print('Deconfounding for sex and age!')
    conf_mat = np.hstack([
        age, age2, sex, sex_x_age, sex_x_age2,
        ])
    
    return conf_mat

In [ ]:
#Deconfounding for sex and age 
#HIPPOCAMPUS 
col_HC.append('PSCID_left')
hc_infos = BL00_HC[col_HC].merge(infos, how='left',left_on='PSCID_left', right_on='PSCID')
print(hc_infos.shape)
conf_mat_hc = deconf(hc_infos)
print(conf_mat_hc.shape)
X_scaler = StandardScaler()
col_HC.remove('PSCID_left')
X = X_scaler.fit_transform(BL00_HC[col_HC])
print(X.shape)
hc = clean(X, confounds=conf_mat_hc, detrend=False, standardize=False)
print(hc.shape)

In [ ]:
col_HC

In [ ]:
BL00_HC.index

In [ ]:
pd.DataFrame(hc, columns=col_HC).set_index(BL00_HC.index).to_csv('BL00_HC_deconf.csv')

In [ ]:
BL00_DN['PSCID'] = BL00_DN.index

In [ ]:
BL00_DN.shape

In [ ]:
#Deconfounding for sex and age 
#DEFAULT NETWORK
dn_infos = BL00_DN.merge(infos, how='left',left_on='PSCID', right_on='PSCID')
print(dn_infos.shape)
conf_mat_dn = deconf(dn_infos)
print(conf_mat_dn.shape)
X_scaler = StandardScaler()
BL00_DN = BL00_DN.drop(columns='PSCID')
X = X_scaler.fit_transform(BL00_DN)
print(X.shape)
dn = clean(X, confounds=conf_mat_dn, detrend=False, standardize=False)
print(dn.shape)

In [ ]:
pd.DataFrame(dn, columns=col_DN).set_index(BL00_DN.index)

In [ ]:
# get x_scores and y_scores from PreventAD
pop_cca = joblib.load('org_CCA_k25_lonely_full')
hc, dn = pop_cca.transform(hc,dn)

In [ ]:
hc.shape

In [ ]:
dn.shape

In [ ]:
BL00_HC.index.T.shape

In [ ]:
cca_BL00 = pd.DataFrame(
#stacking data in a single numpy array    
    np.vstack((
        BL00_HC.index.T, hc.T, dn.T))).T

In [ ]:
cca_BL00

In [ ]:
cca_BL00.to_csv('BL00_CCA_modes.csv')

# FU12

In [ ]:
FU12_HC

In [ ]:
FU12_DN

In [ ]:
#removing two participants without HC volumes based on index
#FU12_DN = FU12_DN.iloc[np.where(FU12_DN.index.isin(FU12_HC[col_HC].index))]

In [ ]:
#Deconfounding for sex and age 
#HIPPOCAMPUS 
col_HC.append('PSCID_left')
hc_infos = FU12_HC[col_HC].merge(infos, how='left',left_on='PSCID_left', right_on='PSCID')
print(hc_infos.shape)
conf_mat_hc = deconf(hc_infos)
print(conf_mat_hc.shape)
X_scaler = StandardScaler()
col_HC.remove('PSCID_left')
X = X_scaler.fit_transform(FU12_HC[col_HC])
print(X.shape)
hc = clean(X, confounds=conf_mat_hc, detrend=False, standardize=False)
print(hc.shape)

In [ ]:
pd.DataFrame(hc, columns=col_HC).set_index(FU12_HC.index).to_csv('FU12_HC_deconf.csv')

In [ ]:
#Deconfounding for sex and age 
#DEFAULT NETWORK
FU12_DN['PSCID'] = FU12_DN.index
dn_infos = FU12_DN.merge(infos, how='left',left_on='PSCID', right_on='PSCID')
print(dn_infos.shape)
conf_mat_dn = deconf(dn_infos)
print(conf_mat_dn.shape)
X_scaler = StandardScaler()
FU12_DN = FU12_DN.drop(columns='PSCID')
X = X_scaler.fit_transform(FU12_DN)
print(X.shape)
dn = clean(X, confounds=conf_mat_dn, detrend=False, standardize=False)
print(dn.shape)

In [ ]:
pd.DataFrame(dn, columns=col_DN).set_index(FU12_DN.index).to_csv('FU12_DN_deconf.csv')

In [ ]:
# get x_scores and y_scores from PreventAD
pop_cca = joblib.load('org_CCA_k25_lonely_full')
hc, dn = pop_cca.transform(hc,dn)

In [ ]:
cca_FU12 = pd.DataFrame(
#stacking data in a single numpy array    
    np.vstack((
        FU12_HC.index.T, hc.T, dn.T))).T
cca_FU12.to_csv('FU12_CCA_modes.csv')

# FU24

In [ ]:
FU24_HC

In [ ]:
FU24_DN

In [ ]:
#removing two participants without HC volumes based on index
#FU24_DN = FU24_DN.iloc[np.where(FU24_DN.index.isin(FU24_HC[col_HC].index))]

In [ ]:
#Deconfounding for sex and age 
#HIPPOCAMPUS 
col_HC.append('PSCID_left')
hc_infos = FU24_HC[col_HC].merge(infos, how='left',left_on='PSCID_left', right_on='PSCID')
print(hc_infos.shape)
conf_mat_hc = deconf(hc_infos)
print(conf_mat_hc.shape)
X_scaler = StandardScaler()
col_HC.remove('PSCID_left')
X = X_scaler.fit_transform(FU24_HC[col_HC])
print(X.shape)
hc = clean(X, confounds=conf_mat_hc, detrend=False, standardize=False)
print(hc.shape)

In [ ]:
pd.DataFrame(hc, columns=col_HC).set_index(FU24_HC.index).to_csv('FU24_HC_deconf.csv')

In [ ]:
#Deconfounding for sex and age 
#DEFAULT NETWORK
FU24_DN['PSCID'] = FU24_DN.index
dn_infos = FU24_DN.merge(infos, how='left',left_on='PSCID', right_on='PSCID')
print(dn_infos.shape)
conf_mat_dn = deconf(dn_infos)
print(conf_mat_dn.shape)
X_scaler = StandardScaler()
FU24_DN = FU24_DN.drop(columns='PSCID')
X = X_scaler.fit_transform(FU24_DN)
print(X.shape)
dn = clean(X, confounds=conf_mat_dn, detrend=False, standardize=False)
print(dn.shape)

In [ ]:
pd.DataFrame(dn, columns=col_DN).set_index(FU24_DN.index).to_csv('FU24_DN_deconf.csv')

In [ ]:
hc.shape

In [ ]:
dn.shape

In [ ]:
pop_cca

In [ ]:
# get x_scores and y_scores from PreventAD
hc, dn = pop_cca.transform(hc,dn)

cca_FU24 = pd.DataFrame(
#stacking data in a single numpy array    
    np.vstack((
        FU24_HC.index.T, hc.T, dn.T))).T
cca_FU24.to_csv('FU24_CCA_modes.csv')

# FU36

In [ ]:
FU36_HC

In [ ]:
FU36_HC['PSCID'] = FU36_HC.index

In [ ]:
FU36_DN

In [ ]:
#removing two participants without HC volumes based on index
#FU36_DN = FU36_DN.iloc[np.where(FU36_DN.index.isin(FU36_HC[col_HC].index))]

In [ ]:
#col_HC.remove('PSCID_left')
#col_HC.remove('PSCID')

In [ ]:
#Deconfounding for sex and age 
#HIPPOCAMPUS 
col_HC.append('PSCID')
hc_infos = FU36_HC[col_HC].merge(infos, how='left',left_on='PSCID', right_on='PSCID')
print(hc_infos.shape)
conf_mat_hc = deconf(hc_infos)
print(conf_mat_hc.shape)
X_scaler = StandardScaler()
col_HC.remove('PSCID')
X = X_scaler.fit_transform(FU36_HC[col_HC])
print(X.shape)
hc = clean(X, confounds=conf_mat_hc, detrend=False, standardize=False)
print(hc.shape)

In [ ]:
pd.DataFrame(hc, columns=col_HC).set_index(FU36_HC.index).to_csv('FU36_HC_deconf.csv')

In [ ]:
#Deconfounding for sex and age 
#DEFAULT NETWORK
FU36_DN['PSCID'] = FU36_DN.index
dn_infos = FU36_DN.merge(infos, how='left',left_on='PSCID', right_on='PSCID')
print(dn_infos.shape)
conf_mat_dn = deconf(dn_infos)
print(conf_mat_dn.shape)
X_scaler = StandardScaler()
FU36_DN = FU36_DN.drop(columns='PSCID')
X = X_scaler.fit_transform(FU36_DN)
print(X.shape)
dn = clean(X, confounds=conf_mat_dn, detrend=False, standardize=False)
print(dn.shape)

In [ ]:
pd.DataFrame(dn, columns=col_DN).set_index(FU36_DN.index).to_csv('FU36_DN_deconf.csv')

In [ ]:
# get x_scores and y_scores from PreventAD
hc, dn = pop_cca.transform(hc,dn)

cca_FU36 = pd.DataFrame(
#stacking data in a single numpy array    
    np.vstack((
        FU36_HC.index.T, hc.T, dn.T))).T
cca_FU36.to_csv('FU36_CCA_modes.csv')

# FU48

In [ ]:
FU48_HC

In [ ]:
FU48_DN

In [ ]:
#removing two participants without HC volumes based on index
FU48_DN = FU48_DN.iloc[np.where(FU48_DN.index.isin(FU48_HC[col_HC].index))]

In [ ]:
FU48_HC.columns

In [ ]:
#Deconfounding for sex and age 
#HIPPOCAMPUS 
FU48_HC['PSCID'] = FU48_HC.index
col_HC.append('PSCID')
hc_infos = FU48_HC[col_HC].merge(infos, how='left',left_on='PSCID', right_on='PSCID')
print(hc_infos.shape)
conf_mat_hc = deconf(hc_infos)
print(conf_mat_hc.shape)
X_scaler = StandardScaler()
col_HC.remove('PSCID')
X = X_scaler.fit_transform(FU48_HC[col_HC])
print(X.shape)
hc = clean(X, confounds=conf_mat_hc, detrend=False, standardize=False)
print(hc.shape)

In [ ]:
pd.DataFrame(hc, columns=col_HC).set_index(FU48_HC.index).to_csv('FU48_HC_deconf.csv')

In [ ]:
#Deconfounding for sex and age 
#DEFAULT NETWORK
FU48_DN['PSCID'] = FU48_DN.index
dn_infos = FU48_DN.merge(infos, how='left',left_on='PSCID', right_on='PSCID')
print(dn_infos.shape)
conf_mat_dn = deconf(dn_infos)
print(conf_mat_dn.shape)
X_scaler = StandardScaler()
FU48_DN = FU48_DN.drop(columns='PSCID')
X = X_scaler.fit_transform(FU48_DN)
print(X.shape)
dn = clean(X, confounds=conf_mat_dn, detrend=False, standardize=False)
print(dn.shape)

In [ ]:
pd.DataFrame(dn, columns=col_DN).set_index(FU48_DN.index).to_csv('FU48_DN_deconf.csv')

In [ ]:
# get x_scores and y_scores from PreventAD
hc, dn = pop_cca.transform(hc,dn)

cca_FU48 = pd.DataFrame(
#stacking data in a single numpy array    
    np.vstack((
        FU48_HC.index.T, hc.T, dn.T))).T
cca_FU48.to_csv('FU48_CCA_modes.csv')